In [1]:
import pandas
import time
import matplotlib.pyplot as plt
import numpy
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

# Graetz and Michaels 2018 - employment share

In [2]:
myeuklems = pandas.read_csv('C:/Ainhoa/Macro-Data/myeuklems.txt',sep=',',index_col=0)
myifr = pandas.read_csv('C:/Ainhoa/Macro-Data/myifr.txt',index_col=0)
myipums1 = pandas.read_csv('C:/Ainhoa/Macro-Data/myipums2.txt',index_col=0)
mydata = myifr.merge(myeuklems,on=['Industry-label','Year'],how='inner')
mydata = mydata.merge(myipums1,on=['Industry-label','Year'],how='inner')
mydata

,Year,Industry-label,Installations,Operational stock,comp,empe,h_empe,i_ct,i_cult,i_gfcf,...,i_omach,i_rd,i_rstruc,i_soft_db,i_traeq,va_cp,va_pi,High-skill occupations,Low-skill occupations,Other occupations
0,2004,Chemical and other mineral,1146,1146,156600.0,2308900.0,4.635382e+06,1052.0,0,93888.0,...,28778.0,52132.0,0.0,3458.0,1354.0,427700.0,187.039236,0.087579,0.467793,0.444628
1,2004,Electronics,1307,1307,148200.0,1767900.0,3.416605e+06,1322.0,0,68863.0,...,12078.0,43088.0,0.0,7903.0,301.0,236800.0,270.257000,0.266830,0.364716,0.368454
2,2004,Food products,628,628,78900.0,1688400.0,3.208441e+06,231.0,0,19530.0,...,11859.0,2703.0,0.0,1319.0,401.0,156300.0,77.161000,0.036007,0.562343,0.401650
3,2004,Machinery,0,0,71800.0,1145200.0,2.307567e+06,218.0,0,14279.0,...,5359.0,5832.0,0.0,1457.0,254.0,101400.0,86.265000,0.120798,0.525706,0.353495
4,2004,Metal,1378,1378,107300.0,1963900.0,3.923814e+06,228.0,0,17053.0,...,10304.0,3545.0,0.0,792.0,497.0,165500.0,82.188326,0.062739,0.627820,0.309441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2020,Machinery,309,5032,96500.0,1055000.0,2.061569e+06,164.0,0,27383.0,...,8388.0,13711.0,0.0,2146.0,226.0,163400.0,123.624000,0.168688,0.463114,0.368199
132,2020,Metal,1985,20959,133100.0,1739500.0,3.413286e+06,237.0,0,32240.0,...,17715.0,8335.0,0.0,1317.0,419.0,212100.0,98.137596,0.083617,0.582145,0.334239
133,2020,Other manufacturing,286,1870,81500.0,1126100.0,2.213306e+06,95.0,0,20077.0,...,12887.0,2949.0,0.0,1310.0,193.0,143900.0,119.964416,0.100010,0.476442,0.423548
134,2020,Textiles,42,292,18300.0,313500.0,6.168384e+05,25.0,0,3177.0,...,1237.0,1310.0,0.0,272.0,8.0,24800.0,110.724505,0.061472,0.568345,0.370183


# Robot density as the stock of robots per million hours worked

In [3]:
mydata = mydata.copy()
mydata['#Robots/H'] = 1000*mydata['Operational stock']/mydata['h_empe']

# Controls

In [4]:
mydata = mydata.copy()
mydata['K/wage'] = mydata['i_gfcf']/mydata['comp']
mydata['ICT/K'] = (mydata['i_it']+mydata['i_ct']+mydata['i_soft_db'])/mydata['i_gfcf']

# Differences - 1 year

In [7]:
mydf = mydata.sort_values(by=['Industry-label','Year'])
mydf = mydf[['Industry-label','Year','High-skill occupations','Low-skill occupations','Other occupations','#Robots/H','K/wage','ICT/K','h_empe']]
mydf['High-skill occupations'] = mydf.groupby(['Industry-label'])['High-skill occupations'].diff()
mydf['Low-skill occupations'] = mydf.groupby(['Industry-label'])['Low-skill occupations'].diff()
mydf['Other occupations'] = mydf.groupby(['Industry-label'])['Other occupations'].diff()
mydf['#Robots/H'] = mydf.groupby(['Industry-label'])['#Robots/H'].diff()
mydf['K/wage'] = mydf.groupby(['Industry-label'])['K/wage'].diff()
mydf['ICT/K'] = mydf.groupby(['Industry-label'])['ICT/K'].diff()
mydf = mydf.dropna()
mydf

,Industry-label,Year,High-skill occupations,Low-skill occupations,Other occupations,#Robots/H,K/wage,ICT/K,h_empe
8,Chemical and other mineral,2005,0.001629,-0.015987,0.014358,0.353475,0.031853,-1.836439e-03,4.529687e+06
16,Chemical and other mineral,2006,0.000760,0.007565,-0.008325,0.373030,0.030236,-8.116969e-03,4.570039e+06
24,Chemical and other mineral,2007,-0.001151,-0.002195,0.003346,0.364771,0.154321,-3.365094e-03,4.493075e+06
32,Chemical and other mineral,2008,0.004977,-0.021203,0.016226,0.377907,-0.003953,5.394018e-03,4.322391e+06
40,Chemical and other mineral,2009,0.000211,-0.007108,0.006896,0.480685,-0.009776,8.391616e-03,3.805022e+06
...,...,...,...,...,...,...,...,...,...
103,Transport equipment,2016,0.010070,-0.006365,-0.003704,1.584020,-0.013442,7.813611e-04,3.366144e+06
111,Transport equipment,2017,0.001253,-0.011522,0.010269,0.616620,0.032161,5.901816e-03,3.373123e+06
119,Transport equipment,2018,0.007756,-0.004186,-0.003570,0.785555,-0.006281,1.213320e-08,3.514151e+06
127,Transport equipment,2019,0.010706,-0.000536,-0.010170,1.345182,-0.067425,1.356415e-02,3.491445e+06


# Share of employment by industry for each year

In [8]:
total = mydf.groupby('Year')['h_empe'].transform('sum')
mydf['share'] = mydf['h_empe'] / total

# Percentile

In [9]:
mydf['Percentile'] = mydf['#Robots/H'].apply(lambda x: stats.percentileofscore(mydf['#Robots/H'],x))
mydf

,Industry-label,Year,High-skill occupations,Low-skill occupations,Other occupations,#Robots/H,K/wage,ICT/K,h_empe,share,Percentile
8,Chemical and other mineral,2005,0.001629,-0.015987,0.014358,0.353475,0.031853,-1.836439e-03,4.529687e+06,0.178440,57.81250
16,Chemical and other mineral,2006,0.000760,0.007565,-0.008325,0.373030,0.030236,-8.116969e-03,4.570039e+06,0.178628,60.15625
24,Chemical and other mineral,2007,-0.001151,-0.002195,0.003346,0.364771,0.154321,-3.365094e-03,4.493075e+06,0.178444,59.37500
32,Chemical and other mineral,2008,0.004977,-0.021203,0.016226,0.377907,-0.003953,5.394018e-03,4.322391e+06,0.178601,60.93750
40,Chemical and other mineral,2009,0.000211,-0.007108,0.006896,0.480685,-0.009776,8.391616e-03,3.805022e+06,0.181710,68.75000
...,...,...,...,...,...,...,...,...,...,...,...
103,Transport equipment,2016,0.010070,-0.006365,-0.003704,1.584020,-0.013442,7.813611e-04,3.366144e+06,0.147572,89.06250
111,Transport equipment,2017,0.001253,-0.011522,0.010269,0.616620,0.032161,5.901816e-03,3.373123e+06,0.146476,76.56250
119,Transport equipment,2018,0.007756,-0.004186,-0.003570,0.785555,-0.006281,1.213320e-08,3.514151e+06,0.148759,79.68750
127,Transport equipment,2019,0.010706,-0.000536,-0.010170,1.345182,-0.067425,1.356415e-02,3.491445e+06,0.148340,85.93750


# Regression for high-skill occupations

In [10]:
df = mydf[['High-skill occupations','Percentile','K/wage','ICT/K','share','Year']]
df = df.copy()
df['Percentile'] = df['Percentile']/100
cluster_var = mydf['Industry-label'].astype(str) + '_' + mydf['Year'].astype(str)
y_dummies = pandas.get_dummies(df['Year'])
df = pandas.concat([df,y_dummies],axis=1)
df.drop([2005,'Year'],axis=1,inplace=True)
Y = df[['High-skill occupations']]
X = df.drop(['High-skill occupations','share'],axis=1)

In [11]:
mod_wls = sm.WLS(Y,X,weights=df['share'])
res_wls = mod_wls.fit(cov_type='cluster',cov_kwds={'groups':cluster_var},use_t=True)
print(res_wls.summary())

                                   WLS Regression Results                                  
Dep. Variable:     High-skill occupations   R-squared (uncentered):                   0.482
Model:                                WLS   Adj. R-squared (uncentered):              0.398
Method:                     Least Squares   F-statistic:                              6.254
Date:                    Tue, 01 Aug 2023   Prob (F-statistic):                    9.77e-11
Time:                            08:07:34   Log-Likelihood:                          497.96
No. Observations:                     128   AIC:                                     -959.9
Df Residuals:                         110   BIC:                                     -908.6
Df Model:                              18                                                  
Covariance Type:                  cluster                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-

# Regression for low-skill occupations

In [12]:
df = mydf[['Low-skill occupations','Percentile','K/wage','ICT/K','share','Year']]
df = df.copy()
df['Percentile'] = df['Percentile']/100
cluster_var = mydf['Industry-label'].astype(str) + '_' + mydf['Year'].astype(str)
y_dummies = pandas.get_dummies(df['Year'])
df = pandas.concat([df,y_dummies],axis=1)
df.drop([2005,'Year'],axis=1,inplace=True)
Y = df[['Low-skill occupations']]
X = df.drop(['Low-skill occupations','share'],axis=1)

In [13]:
mod_wls = sm.WLS(Y,X,weights=df['share'])
res_wls = mod_wls.fit(cov_type='cluster',cov_kwds={'groups':cluster_var},use_t=True)
print(res_wls.summary())

                                  WLS Regression Results                                  
Dep. Variable:     Low-skill occupations   R-squared (uncentered):                   0.395
Model:                               WLS   Adj. R-squared (uncentered):              0.297
Method:                    Least Squares   F-statistic:                              5.457
Date:                   Tue, 01 Aug 2023   Prob (F-statistic):                    2.77e-09
Time:                           08:08:43   Log-Likelihood:                          426.26
No. Observations:                    128   AIC:                                     -816.5
Df Residuals:                        110   BIC:                                     -765.2
Df Model:                             18                                                  
Covariance Type:                 cluster                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------

# Regression for other occupations

In [14]:
df = mydf[['Other occupations','Percentile','K/wage','ICT/K','share','Year']]
df = df.copy()
df['Percentile'] = df['Percentile']/100
cluster_var = mydf['Industry-label'].astype(str) + '_' + mydf['Year'].astype(str)
y_dummies = pandas.get_dummies(df['Year'])
df = pandas.concat([df,y_dummies],axis=1)
df.drop([2005,'Year'],axis=1,inplace=True)
Y = df[['Other occupations']]
X = df.drop(['Other occupations','share'],axis=1)

In [15]:
mod_wls = sm.WLS(Y,X,weights=df['share'])
res_wls = mod_wls.fit(cov_type='cluster',cov_kwds={'groups':cluster_var},use_t=True)
print(res_wls.summary())

                                 WLS Regression Results                                
Dep. Variable:      Other occupations   R-squared (uncentered):                   0.257
Model:                            WLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              3.016
Date:                Tue, 01 Aug 2023   Prob (F-statistic):                    0.000154
Time:                        08:09:55   Log-Likelihood:                          439.04
No. Observations:                 128   AIC:                                     -842.1
Df Residuals:                     110   BIC:                                     -790.7
Df Model:                          18                                                  
Covariance Type:              cluster                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------